# 1b) Data Collection Updater - US Streaming Hits

Running the codes in part 1a) takes an extremely long amount of time. We've reached a point where the dataset is 2500+ songs large, and the Spotify API almost always crashes when trying to run the codes in part 1a. Trying to run the code to retrieve metadata on all 2500+ songs gives us an error of too many API requests being made.

 Two alternatives exist to account for this

- Alternative 1: Run the code to retrieve data for the entire dataset in chunks, spread out throughout different days. This is not an ideal solution though.
- Alternative 2: Maintain all previously acquired data, and add onto it whenever a new chart is published.

Alternative 2 is far more reasonable; therefore, alternative 2 will be chosen. The following Jupyter Notebook retrieves previously collected data, and adds newly collected data as new weekly chart data is published by HITS Daily Double and Billboard.

## Imports
First, install the following (using "pip install _insert_package_here_").
These packages are to be used in the following notebooks as well.
- [spotipy](https://spotipy.readthedocs.io/en/master/)
- pandas
- [gspread](https://docs.gspread.org/en/latest/)
- oauth2client
- [df2gspread](https://df2gspread.readthedocs.io/en/latest/)
- matplotlib
- seaborn
- sklearn

Next, we import the following for this notebook:

In [1]:
#to access spotipy
import configparser
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

#to allow interaction with underlying operating system
import os

#to parse through spotipy's output
import json

#for dataframe cleaning
import numpy as np
import pandas as pd

#to access google sheets with python
import gspread
from google.oauth2 import service_account
from oauth2client.service_account import ServiceAccountCredentials

#to edit google sheets from python
from df2gspread import df2gspread as d2g

#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

#to remove the limits on a dataframe's columns displayed
pd.set_option('display.max_columns', None) 

#import spotify client credentials 

# Spotify API Client Credentials
First, ensure that you have a [Spotify For Developers](https://developer.spotify.com) account. Then, using [this link](https://developer.spotify.com/dashboard/applications) (under dashboards then applications), create an new app and [get your client ID and client secret ID](https://developer.spotify.com/documentation/general/guides/authorization/app-settings/). 

It is __always good practice to hide passwords from code__. Therefore, [__environmental variables__ are to be created](https://softhints.com/how-to-set-environmental-variable-in-jupyter-notebook/) in the following:

##  CONVERT THE FOLLOWING TO CODE (W/ YOUR OWN SPOTIFY CREDENTIALS):

- The following creates environmental variables:

%env SPOTIPY_CLIENT_ID=YOUR_SPOTIFY_APP_CLIENT_ID

%env SPOTIPY_CLIENT_SECRET=YOUR_SPOTIFY_APP_CLIENT_SECRET_ID

%env SPOTIPY_REDIRECT_URI=http://127.0.0.1:8080/
        
___my code and its output is hidden to protect privacy___

In [3]:
#import spotify client credentials 
spotify_client_id = os.environ['SPOTIPY_CLIENT_ID']
spotify_secret = os.environ['SPOTIPY_CLIENT_SECRET']
spotify_redirect_uri = os.environ['SPOTIPY_REDIRECT_URI']

#create a Spotify instance
client_credentials_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

#to confirm 
print(sp)

# Importing Google Sheets

Use song information from [this Google Sheet](https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit?usp=sharing). 

Specifically we import data from the following sheets:
- 1) "HITS Streaming Songs" - This sheet collects all hit songs from HITS Daily Double, a music industry trade publication, and their weekly Revenue Songs chart. Occasionially, data from HITS Daily Double's Streaming Songs chart are used, alongside with calculations made utilizing Billboard's Streaming Songs chart. The data includes how many times a song was streamed week-to-week for most weeks since 2017. Each week, when a new chart is posted by HITS Daily Double/Billboard, the sheet is manually updated with song information that corresponds to Spotify (song name, artist name, album title). 
- 2) "DF to Gspread" - This sheet collects all Spotify's metadata from each song present in the previous sheet. It is to be updated whenever this Jupyter notebook is ran.

In order to be able to use this data on your own device, create a copy of the Google Sheet to your own Drive. Also, [follow the instructions here](https://github.com/PrettyPrinted/youtube_video_code/tree/master/2021/10/14/How%20to%20Use%20Google%20Sheets%20With%20Python%20(2021)) to connect Google Drive/Google Sheets API with Python.

For this Python notebook, the file for our service account is the json file with the Google API keys downloaded from Google Developers Console. Please reference the link above for more instructions.

## Importing Google Sheet 1

In [4]:
#service account
sa = gspread.service_account(filename="gspread_service_account.json")

#sheet
sh = sa.open("HITS Streaming Songs")

#worksheet
wks = sh.worksheet("HITS Streaming Songs")

#to get all the values in the sheet into a dataframe
gsheets_df = pd.DataFrame(wks.get_all_values())

header = gsheets_df.iloc[0] #isolate first row as header
gsheets_df = gsheets_df[1:] #get rid of header in original df
gsheets_df.columns = header #convert first row to headers

#set 'index' columnn as index
gsheets_df = gsheets_df.set_index('index')

#removing extra whitespace that might be present in title/artist/album titles
gsheets_df['title'] = gsheets_df['title'].str.rstrip(' ')
gsheets_df['artist'] = gsheets_df['artist'].str.rstrip(' ')
gsheets_df['album'] = gsheets_df['album'].str.rstrip(' ')

#convert index from string to int and sort the dataframe by index
gsheets_df.index = gsheets_df.index.astype(int)
gsheets_df = gsheets_df.sort_values('index')

#fill in blank values with 0
gsheets_df.loc[:,'streams_2017_to_present':'playlist_index'] = gsheets_df.loc[:,'streams_2017_to_present':'playlist_index'].replace(r'^\s*$', "0", regex=True)

#convert all numbers-as-strings to numbers from row 4 in python's index and onwards
for col_name in gsheets_df.columns.to_list()[4:len(gsheets_df.columns.to_list())-1]:
    
    #get rid of strings' commas and convert strings to integers
    gsheets_df[col_name] = gsheets_df[col_name].str.replace(',','').astype('int')
    
#turn peak_date strings to date format
gsheets_df['peak_date'] = pd.to_datetime(gsheets_df['peak_date'])

gsheets_df

,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,current_year,peak_week_streams,2024-07-26,2024-07-19,2024-07-12,2024-07-05,2024-06-28,2024-06-21,2024-06-14,2024-06-07,2024-05-31,2024-05-24,2024-05-17,2024-05-10,2024-05-03,2024-04-26,2024-04-19,2024-04-12,2024-04-05,2024-03-29,2024-03-22,2024-03-15,2024-03-08,2024-03-01,2024-02-23,2024-02-16,2024-02-09,2024-02-02,2024-01-26,2024-01-19,2024-01-12,2024-01-05,2023-12-29,2023-12-22,2023-12-15,2023-12-08,2023-12-01,2023-11-24,2023-11-17,2023-11-10,2023-11-03,2023-10-27,2023-10-20,2023-10-13,2023-10-06,2023-09-29,2023-09-22,2023-09-15,2023-09-08,2023-09-01,2023-08-25,2023-08-18,2023-08-11,2023-08-04,2023-07-28,2023-07-21,2023-07-14,2023-07-07,2023-06-30,2023-06-23,2023-06-16,2023-06-09,2023-06-02,2023-05-26,2023-05-19,2023-05-12,2023-05-05,2023-04-28,2023-04-21,2023-04-14,2023-04-07,2023-03-31,2023-03-24,2023-03-17,2023-03-10,2023-03-03,2023-02-24,2023-02-17,2023-02-10,2023-02-03,2023-01-27,2023-01-20,2023-01-13,2023-01-06,2022-12-30,2022-12-23,2022-12-16,2022-12-09,2022-12-02,2022-11-25,2022-11-18,2022-11-11,2022-11-04,2022-10-28,2022-10-21,2022-10-14,2022-10-07,2022-09-30,2022-09-23,2022-09-16,2022-09-09,2022-09-02,2022-08-26,2022-08-19,2022-08-12,2022-08-05,2022-07-29,2022-07-22,2022-07-15,2022-07-08,2022-07-01,2022-06-24,2022-06-17,2022-06-10,2022-06-03,2022-05-27,2022-05-20,2022-05-13,2022-05-06,2022-04-29,2022-04-22,2022-04-15,2022-04-08,2022-04-01,2022-03-25,2022-03-18,2022-03-11,2022-03-04,2022-02-25,2022-02-18,2022-02-11,2022-02-04,2022-01-28,2022-01-21,2022-01-14,2022-01-07,2021-12-31,2021-12-24,2021-12-17,2021-12-10,2021-12-03,2021-11-26,2021-11-19,2021-11-12,2021-11-05,2021-10-29,2021-10-22,2021-10-15,2021-10-08,2021-10-01,2021-09-24,2021-09-17,2021-09-10,2021-09-03,2021-08-27,2021-08-20,2021-08-13,2021-08-06,2021-07-30,2021-07-23,2021-07-16,2021-07-09,2021-07-02,2021-06-25,2021-06-18,2021-06-11,2021-06-04,2021-05-28,2021-05-21,2021-05-14,2021-05-07,2021-04-30,2021-04-23,2021-04-16,2021-04-09,2021-04-02,2021-03-26,2021-03-19,2021-03-12,2021-03-05,2021-02-26,2021-02-19,2021-02-12,2021-02-05,2021-01-29,2021-01-22,2021-01-15,2021-01-08,2021-01-01,2020-12-25,2020-12-18,2020-12-11,2020-12-04,2020-11-27,2020-11-20,2020-11-13,2020-11-06,2020-10-30,2020-10-23,2020-10-16,2020-10-09,2020-10-02,2020-09-25,2020-09-18,2020-09-11,2020-09-04,2020-08-28,2020-08-21,2020-08-14,2020-08-07,2020-07-31,2020-07-24,2020-07-17,2020-07-10,2020-07-03,2020-06-26,2020-06-19,2020-06-12,2020-06-05,2020-05-29,2020-05-22,2020-05-15,2020-05-08,2020-05-01,2020-04-24,2020-04-17,2020-04-10,2020-04-03,2020-03-27,2020-03-20,2020-03-13,2020-03-06,2020-02-28,2020-02-21,2020-02-14,2020-02-07,2020-01-31,2020-01-24,2020-01-17,2020-01-10,2020-01-03,2019-12-27,2019-12-20,2019-12-13,2019-12-06,2019-11-29,2019-11-22,2019-11-15,2019-11-08,2019-11-01,2019-10-25,2019-10-18,2019-10-11,2019-10-04,2019-09-27,2019-09-20,2019-09-13,2019-09-06,2019-08-30,2019-08-23,2019-08-16,2019-08-09,2019-08-02,2019-07-26,2019-07-19,2019-07-12,2019-07-05,2019-06-28,2019-06-21,2019-06-14,2019-06-07,2019-05-31,2019-05-24,2019-05-17,2019-05-10,2019-05-03,2019-04-26,2019-04-19,2019-04-12,2019-04-05,2019-03-29,2019-03-22,2019-03-15,2019-03-08,2019-03-01,2019-02-22,2019-02-15,2019-02-08,2019-02-01,2019-01-25,2019-01-18,2019-01-11,2019-01-04,2018-12-28,2018-12-21,2018-12-14,2018-12-07,2018-11-30,2018-11-23,2018-11-16,2018-11-09,2018-11-02,2018-10-26,2018-10-19,2018-10-12,2018-10-05,2018-09-28,2018-09-21,2018-09-14,2018-09-07,2018-08-31,2018-08-24,2018-08-17,2018-08-10,2018-08-03,2018-07-27,2018-07-20,2018-07-13,2018-07-06,2018-06-29,2018-06-22,2018-06-15,2018-06-08,2018-06-01,2018-05-25,2018-05-18,2018-05-11,2018-05-04,2018-04-27,2018-04-20,2018-04-13,2018-04-06,2018-03-30,2018-03-23,2018-03-16,2018-03-09,2018-03-02,2018-02-23,2018-02-16,2018-02-09,2018-02-02,2018-01-26,2018-01-19,2018-01-12,2018-01-05,2017-12-29,2017-12-15,2017-12-08,2017-12-01,2017-11-24,2017-11-17,2017-11-10,2017-11-03,2017-10-27,2017-10-20,2017-10-13,2

## Importing Google Sheet 2

In [5]:
#dg to gspread worksheet
df2gspread_wks = sh.worksheet("DF to Gspread")

#uses all values in the worksheet for the data frame
df2gspread_wks = pd.DataFrame(df2gspread_wks.get_all_values())

header = df2gspread_wks.iloc[0] #isolate first row as header
df2gspread_wks = df2gspread_wks[1:] #get rid of header in original df
df2gspread_wks.columns = header

df2gspread_wks = df2gspread_wks.set_index('index') #set 'index' as index

#convert index from string to int
df2gspread_wks.index = df2gspread_wks.index.astype(int)

#convert all numbers-as-strings to numbers
df2gspread_wks.loc[:,'streams_2017_to_present':'playlist_index'] = df2gspread_wks.loc[:,'streams_2017_to_present':'playlist_index'].replace(r'^\s*$', "0", regex=True)

#turn peak_date strings to date format
df2gspread_wks['peak_date'] = pd.to_datetime(df2gspread_wks['peak_date'])
df2gspread_wks['release_date'] = pd.to_datetime(df2gspread_wks['release_date'])

#change the following columns to int/float format
df2gspread_wks.loc[:,'streams_2017_to_present':'playlist_index'] = df2gspread_wks.loc[:,'streams_2017_to_present':'playlist_index'].astype(int)
df2gspread_wks.loc[:,'length'] = df2gspread_wks.loc[:,'length'].astype(float)
df2gspread_wks.loc[:,'length':'valence'] = df2gspread_wks.loc[:,'length':'valence'].astype(float)

df2gspread_wks

,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,current_year,peak_week_streams,2024-07-19,2024-07-12,2024-07-05,2024-06-28,2024-06-21,2024-06-14,2024-06-07,2024-05-31,2024-05-24,2024-05-17,2024-05-10,2024-05-03,2024-04-26,2024-04-19,2024-04-12,2024-04-05,2024-03-29,2024-03-22,2024-03-15,2024-03-08,2024-03-01,2024-02-23,2024-02-16,2024-02-09,2024-02-02,2024-01-26,2024-01-19,2024-01-12,2024-01-05,2023-12-29,2023-12-22,2023-12-15,2023-12-08,2023-12-01,2023-11-24,2023-11-17,2023-11-10,2023-11-03,2023-10-27,2023-10-20,2023-10-13,2023-10-06,2023-09-29,2023-09-22,2023-09-15,2023-09-08,2023-09-01,2023-08-25,2023-08-18,2023-08-11,2023-08-04,2023-07-28,2023-07-21,2023-07-14,2023-07-07,2023-06-30,2023-06-23,2023-06-16,2023-06-09,2023-06-02,2023-05-26,2023-05-19,2023-05-12,2023-05-05,2023-04-28,2023-04-21,2023-04-14,2023-04-07,2023-03-31,2023-03-24,2023-03-17,2023-03-10,2023-03-03,2023-02-24,2023-02-17,2023-02-10,2023-02-03,2023-01-27,2023-01-20,2023-01-13,2023-01-06,2022-12-30,2022-12-23,2022-12-16,2022-12-09,2022-12-02,2022-11-25,2022-11-18,2022-11-11,2022-11-04,2022-10-28,2022-10-21,2022-10-14,2022-10-07,2022-09-30,2022-09-23,2022-09-16,2022-09-09,2022-09-02,2022-08-26,2022-08-19,2022-08-12,2022-08-05,2022-07-29,2022-07-22,2022-07-15,2022-07-08,2022-07-01,2022-06-24,2022-06-17,2022-06-10,2022-06-03,2022-05-27,2022-05-20,2022-05-13,2022-05-06,2022-04-29,2022-04-22,2022-04-15,2022-04-08,2022-04-01,2022-03-25,2022-03-18,2022-03-11,2022-03-04,2022-02-25,2022-02-18,2022-02-11,2022-02-04,2022-01-28,2022-01-21,2022-01-14,2022-01-07,2021-12-31,2021-12-24,2021-12-17,2021-12-10,2021-12-03,2021-11-26,2021-11-19,2021-11-12,2021-11-05,2021-10-29,2021-10-22,2021-10-15,2021-10-08,2021-10-01,2021-09-24,2021-09-17,2021-09-10,2021-09-03,2021-08-27,2021-08-20,2021-08-13,2021-08-06,2021-07-30,2021-07-23,2021-07-16,2021-07-09,2021-07-02,2021-06-25,2021-06-18,2021-06-11,2021-06-04,2021-05-28,2021-05-21,2021-05-14,2021-05-07,2021-04-30,2021-04-23,2021-04-16,2021-04-09,2021-04-02,2021-03-26,2021-03-19,2021-03-12,2021-03-05,2021-02-26,2021-02-19,2021-02-12,2021-02-05,2021-01-29,2021-01-22,2021-01-15,2021-01-08,2021-01-01,2020-12-25,2020-12-18,2020-12-11,2020-12-04,2020-11-27,2020-11-20,2020-11-13,2020-11-06,2020-10-30,2020-10-23,2020-10-16,2020-10-09,2020-10-02,2020-09-25,2020-09-18,2020-09-11,2020-09-04,2020-08-28,2020-08-21,2020-08-14,2020-08-07,2020-07-31,2020-07-24,2020-07-17,2020-07-10,2020-07-03,2020-06-26,2020-06-19,2020-06-12,2020-06-05,2020-05-29,2020-05-22,2020-05-15,2020-05-08,2020-05-01,2020-04-24,2020-04-17,2020-04-10,2020-04-03,2020-03-27,2020-03-20,2020-03-13,2020-03-06,2020-02-28,2020-02-21,2020-02-14,2020-02-07,2020-01-31,2020-01-24,2020-01-17,2020-01-10,2020-01-03,2019-12-27,2019-12-20,2019-12-13,2019-12-06,2019-11-29,2019-11-22,2019-11-15,2019-11-08,2019-11-01,2019-10-25,2019-10-18,2019-10-11,2019-10-04,2019-09-27,2019-09-20,2019-09-13,2019-09-06,2019-08-30,2019-08-23,2019-08-16,2019-08-09,2019-08-02,2019-07-26,2019-07-19,2019-07-12,2019-07-05,2019-06-28,2019-06-21,2019-06-14,2019-06-07,2019-05-31,2019-05-24,2019-05-17,2019-05-10,2019-05-03,2019-04-26,2019-04-19,2019-04-12,2019-04-05,2019-03-29,2019-03-22,2019-03-15,2019-03-08,2019-03-01,2019-02-22,2019-02-15,2019-02-08,2019-02-01,2019-01-25,2019-01-18,2019-01-11,2019-01-04,2018-12-28,2018-12-21,2018-12-14,2018-12-07,2018-11-30,2018-11-23,2018-11-16,2018-11-09,2018-11-02,2018-10-26,2018-10-19,2018-10-12,2018-10-05,2018-09-28,2018-09-21,2018-09-14,2018-09-07,2018-08-31,2018-08-24,2018-08-17,2018-08-10,2018-08-03,2018-07-27,2018-07-20,2018-07-13,2018-07-06,2018-06-29,2018-06-22,2018-06-15,2018-06-08,2018-06-01,2018-05-25,2018-05-18,2018-05-11,2018-05-04,2018-04-27,2018-04-20,2018-04-13,2018-04-06,2018-03-30,2018-03-23,2018-03-16,2018-03-09,2018-03-02,2018-02-23,2018-02-16,2018-02-09,2018-02-02,2018-01-26,2018-01-19,2018-01-12,2018-01-05,2017-12-29,2017-12-15,2017-12-08,2017-12-01,2017-11-24,2017-11-17,2017-11-10,2017-11-03,2017-10-27,2017-10-20,2017-10-13,2017-10-06,2

# Merging tables
Merging tables and gathering songs that either didn't merge or have no URI information

In [6]:
def songs_to_update(df1, df2):
    # Merge the dataframes on "title" and "album" without "_x" suffix
    merged_df = pd.merge(df1, df2, on=["title", "artist",'album'], how="left", indicator=True, suffixes=('', '_right'))
    merged_df = merged_df.reset_index(drop=True)
    merged_df.index = merged_df.index + 1

    # Combine both conditions to find unique rows that are either not in df2 or have a missing 'uri'
    combined_conditions = merged_df[(merged_df["_merge"] == "left_only") | (merged_df['uri'].isna())]

    # Drop the "_merge" column from the result as it's no longer needed
    combined_conditions = combined_conditions.drop(columns=["_merge"])

    return combined_conditions


# combined_conditions contains rows where "title" or "album" are different in both dataframes or where URI is not present
songs_to_update_result = songs_to_update(gsheets_df, df2gspread_wks)
songs_to_update_result#.head(8)

,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,current_year,peak_week_streams,2024-07-26,2024-07-19,2024-07-12,2024-07-05,2024-06-28,2024-06-21,2024-06-14,2024-06-07,2024-05-31,2024-05-24,2024-05-17,2024-05-10,2024-05-03,2024-04-26,2024-04-19,2024-04-12,2024-04-05,2024-03-29,2024-03-22,2024-03-15,2024-03-08,2024-03-01,2024-02-23,2024-02-16,2024-02-09,2024-02-02,2024-01-26,2024-01-19,2024-01-12,2024-01-05,2023-12-29,2023-12-22,2023-12-15,2023-12-08,2023-12-01,2023-11-24,2023-11-17,2023-11-10,2023-11-03,2023-10-27,2023-10-20,2023-10-13,2023-10-06,2023-09-29,2023-09-22,2023-09-15,2023-09-08,2023-09-01,2023-08-25,2023-08-18,2023-08-11,2023-08-04,2023-07-28,2023-07-21,2023-07-14,2023-07-07,2023-06-30,2023-06-23,2023-06-16,2023-06-09,2023-06-02,2023-05-26,2023-05-19,2023-05-12,2023-05-05,2023-04-28,2023-04-21,2023-04-14,2023-04-07,2023-03-31,2023-03-24,2023-03-17,2023-03-10,2023-03-03,2023-02-24,2023-02-17,2023-02-10,2023-02-03,2023-01-27,2023-01-20,2023-01-13,2023-01-06,2022-12-30,2022-12-23,2022-12-16,2022-12-09,2022-12-02,2022-11-25,2022-11-18,2022-11-11,2022-11-04,2022-10-28,2022-10-21,2022-10-14,2022-10-07,2022-09-30,2022-09-23,2022-09-16,2022-09-09,2022-09-02,2022-08-26,2022-08-19,2022-08-12,2022-08-05,2022-07-29,2022-07-22,2022-07-15,2022-07-08,2022-07-01,2022-06-24,2022-06-17,2022-06-10,2022-06-03,2022-05-27,2022-05-20,2022-05-13,2022-05-06,2022-04-29,2022-04-22,2022-04-15,2022-04-08,2022-04-01,2022-03-25,2022-03-18,2022-03-11,2022-03-04,2022-02-25,2022-02-18,2022-02-11,2022-02-04,2022-01-28,2022-01-21,2022-01-14,2022-01-07,2021-12-31,2021-12-24,2021-12-17,2021-12-10,2021-12-03,2021-11-26,2021-11-19,2021-11-12,2021-11-05,2021-10-29,2021-10-22,2021-10-15,2021-10-08,2021-10-01,2021-09-24,2021-09-17,2021-09-10,2021-09-03,2021-08-27,2021-08-20,2021-08-13,2021-08-06,2021-07-30,2021-07-23,2021-07-16,2021-07-09,2021-07-02,2021-06-25,2021-06-18,2021-06-11,2021-06-04,2021-05-28,2021-05-21,2021-05-14,2021-05-07,2021-04-30,2021-04-23,2021-04-16,2021-04-09,2021-04-02,2021-03-26,2021-03-19,2021-03-12,2021-03-05,2021-02-26,2021-02-19,2021-02-12,2021-02-05,2021-01-29,2021-01-22,2021-01-15,2021-01-08,2021-01-01,2020-12-25,2020-12-18,2020-12-11,2020-12-04,2020-11-27,2020-11-20,2020-11-13,2020-11-06,2020-10-30,2020-10-23,2020-10-16,2020-10-09,2020-10-02,2020-09-25,2020-09-18,2020-09-11,2020-09-04,2020-08-28,2020-08-21,2020-08-14,2020-08-07,2020-07-31,2020-07-24,2020-07-17,2020-07-10,2020-07-03,2020-06-26,2020-06-19,2020-06-12,2020-06-05,2020-05-29,2020-05-22,2020-05-15,2020-05-08,2020-05-01,2020-04-24,2020-04-17,2020-04-10,2020-04-03,2020-03-27,2020-03-20,2020-03-13,2020-03-06,2020-02-28,2020-02-21,2020-02-14,2020-02-07,2020-01-31,2020-01-24,2020-01-17,2020-01-10,2020-01-03,2019-12-27,2019-12-20,2019-12-13,2019-12-06,2019-11-29,2019-11-22,2019-11-15,2019-11-08,2019-11-01,2019-10-25,2019-10-18,2019-10-11,2019-10-04,2019-09-27,2019-09-20,2019-09-13,2019-09-06,2019-08-30,2019-08-23,2019-08-16,2019-08-09,2019-08-02,2019-07-26,2019-07-19,2019-07-12,2019-07-05,2019-06-28,2019-06-21,2019-06-14,2019-06-07,2019-05-31,2019-05-24,2019-05-17,2019-05-10,2019-05-03,2019-04-26,2019-04-19,2019-04-12,2019-04-05,2019-03-29,2019-03-22,2019-03-15,2019-03-08,2019-03-01,2019-02-22,2019-02-15,2019-02-08,2019-02-01,2019-01-25,2019-01-18,2019-01-11,2019-01-04,2018-12-28,2018-12-21,2018-12-14,2018-12-07,2018-11-30,2018-11-23,2018-11-16,2018-11-09,2018-11-02,2018-10-26,2018-10-19,2018-10-12,2018-10-05,2018-09-28,2018-09-21,2018-09-14,2018-09-07,2018-08-31,2018-08-24,2018-08-17,2018-08-10,2018-08-03,2018-07-27,2018-07-20,2018-07-13,2018-07-06,2018-06-29,2018-06-22,2018-06-15,2018-06-08,2018-06-01,2018-05-25,2018-05-18,2018-05-11,2018-05-04,2018-04-27,2018-04-20,2018-04-13,2018-04-06,2018-03-30,2018-03-23,2018-03-16,2018-03-09,2018-03-02,2018-02-23,2018-02-16,2018-02-09,2018-02-02,2018-01-26,2018-01-19,2018-01-12,2018-01-05,2017-12-29,2017-12-15,2017-12-08,2017-12-01,2017-11-24,2017-11-17,2017-11-10,2017-11-03,2017-10-27,2017-10-20,2017-10-13,2

# Get song URIs from Spotify from list of songs above ⬆️
_parent album - for this project, to be defined as the album that a single comes from_

__A URI (Uniform Resource Indicator) is a unique link to a song/album/playlist on Spotify. The following function takes the song titles, album titles, and artists names from the DataFrame above and uses them as keywords to search for each song's unique URI.__

However, when searching up a song on Spotify, numerous versions of the same song can be returned (single version vs album version, explicit version vs clean version, clean song from a clean parent album or clean song from an explicit parent album). For this analysis (_and for the creation of a playlist on another python document!_), __songs are taken from the explicit version of their parent album/single/EP, regardless of whether the song itself is explicit or clean. If no explicit version of the parent album/single/EP exists, the original clean version is to be used.__

The URIs (and explicit status) for each song in the DataFrame above are to be retrieved using the following functions:

In [7]:
def search_for_URI(data):
    """
    Returns track URIs and the explicit status for a song

        Parameters:
            data (series): series containing a song title, artist, and the song's album
        
        Returns:
            song_info_list (list): list of song title, artist, album title, song URI and explicit status
                #otherwise, returns just the song title, artist, and album title
    """
    song_to_search = data['title'].replace("’","").replace("'","").lower()
    album_to_search = data['album'].replace("’","").replace("'","").lower()
    to_search = song_to_search + " " + data['artist']
    
    #this function searches for q on Spotify
    result = sp.search(q=to_search)
        
    #loops through all results from search
    for song_info_index in range(len(result['tracks']['items'])): 

        #checks for results' album to match the input data's album title (taking into account apostrophe formats)
        if result['tracks']['items'][song_info_index]['album']['name'].replace("’","").replace("'","").lower() == album_to_search:
                        
            #checks for results' song title to match input data's song title (taking into account apostrophe formats)
            if result['tracks']['items'][song_info_index]['name'].replace("’","").replace("'","").lower() == song_to_search:     
                
                #empty list to be filled with name/artist/album/explicit/uri info
                song_info_list = []

                #checks if a song is explicit       
                if result['tracks']['items'][song_info_index]['explicit'] == True:
                    song_info_list.append(result['tracks']['items'][song_info_index]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['artists'][0]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['album']['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['explicit'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['uri'])
                    return song_info_list
                
                #first, if a song is NOT explicit
                if result['tracks']['items'][song_info_index]['explicit'] == False:
                    song_info_list.append(result['tracks']['items'][song_info_index]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['artists'][0]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['album']['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['explicit'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['uri'])
                    return song_info_list
    
    #return just the song title/artist/album title if the URI is not found
    return [data['title'],data['artist'],data['album'],"",""]

In [8]:
def generate_uri_list(songs_to_update_result):
    uri_list = []

    for index, row in songs_to_update_result.iterrows():
        # Call search_for_URI for each row in songs_to_update_result
        result = search_for_URI(row)

        if result:
            # Append the result to the uri_list
            uri_list.append(result)

    return uri_list

# Create a list of lists with URI information
uri_list = generate_uri_list(songs_to_update_result)

# Convert the list of lists into a dataframe
columns = ['title', 'artist', 'album', 'explicit', 'uri']
new_uri_df = pd.DataFrame(uri_list, columns=columns)

# Set the index of new_uri_df to match the index of songs_to_update_result
new_uri_df = new_uri_df.set_index(songs_to_update_result.index)

new_uri_df

,title,artist,album,explicit,uri
3026,Guy For That (Feat. Luke Combs),Post Malone,Guy For That,False,spotify:track:5crxqqibcvoOQQgg4HjAFQ
3027,360,Charli xcx,BRAT,True,spotify:track:4w2GLmK2wnioVnb5CPQeex
3028,Lonely Road (with Jelly Roll),mgk,Lonely Road (with Jelly Roll),False,spotify:track:3Rfre3qkrhwdZZ7dyznwbN


## If necessary, manually find the song
If the function above didn't retrieve a URI successfully, run the following to manually find the song's URI

## Check for songs that don't have a URI

In [9]:
#create a dataframe with all songs that have no URI
data_to_clean = new_uri_df[new_uri_df['uri'] == ""]
data_to_clean.index = range(1,len(data_to_clean)+1)
data_to_clean

,title,artist,album,explicit,uri


# Searching for missing URIs

Use __the following functions to both search for missing URIs and manually input them into the dataframe__:

In [10]:
def song_uri_search(q):
    """
    Prints out different URIs (and other info) from a resulting search on spotipy
    
    Parameter:
        q (str): a string to look up using spotipy's search function
    """
    result = sp.search(q) 
    
    for song_info_dict in range(len(result['tracks']['items'])):
        print("Song: ", result['tracks']['items'][song_info_dict]['name'])
        print("Artist: ", result['tracks']['items'][song_info_dict]['artists'][0]['name'])
        print("Album: ", result['tracks']['items'][song_info_dict]['album']['name'])
        print("Explicit status: ", result['tracks']['items'][song_info_dict]['explicit'])
        print("URI: ", result['tracks']['items'][song_info_dict]['uri'],"\n")

In [11]:
def data_cleaning_func(data_to_clean, uri_df):
    #loop through every song in data_to_clean
    for i in range(0,len(data_to_clean)):
        print("Please follow the following steps to fill in the missing URI(s)." + "\n")
        song_to_search = data_to_clean.iloc[i]['song'].replace("’","").replace("'","")
        album_to_search = data_to_clean.iloc[i]['album'].replace("’","").replace("'","")
        artist_to_search = data_to_clean.iloc[i]['artist']
            
        #concatenate the song title, artist name, and album title to search
        to_search = song_to_search + " " + artist_to_search + " " + album_to_search

        result = sp.search(to_search)
        print("The following results are the search results from searching: '" + to_search + "'")
        print("\n")
            
        #call on song_uri_search function to print out search results
        song_uri_search(to_search)
        print("Search the URI(s) on your browser to double check the song is correct")
            
        #ensure user obtains desired result from the above search
        print("Did you obtain a desired result? Please type 'yes' or 'no': ")
        desired_result = str(input())
            
        #ensure desired_result is a valid input
        while (desired_result != 'yes')  and (desired_result != 'no'):
            print("Please give a valid input. Type either 'yes' or 'no': ")
            desired_result = str(input())
            
        #run the following in case the desired output is not available in the search above
        while desired_result == 'no':
            print("\nPlease manually search the information for this song (be as specific as possible): ")
            manual_search = str(input())
            song_uri_search(manual_search)
                
            #ask if another search is necessary
            print("\nWould you like to make another search? Please type in 'yes' or 'no': ")
            another_search = str(input())
                
            #ensure request for another search is valid
            while (another_search != 'yes')  and (another_search != 'no'):
                print("\nPlease give a valid input. Type either 'yes' or 'no': ")
                another_search = str(input())
                
            if another_search == "no":
                desired_result = 'yes'
            if another_search == "yes":
                desired_result = 'no'
            
        #the index of the song being searched, obtained from uri_df using song and album titles from data_to_clean
        missing_song_index = uri_df[(uri_df['song']==data_to_clean.iloc[i]['song']) & (uri_df['album']==data_to_clean.iloc[i]['album'])].index.values[0]
            
        #obtain explicit status of the song. Have user input the status because numerous songs can result from the search
        print("\nPlease copy and paste the explicit status of your selected song.")
        print("Alternatively, type in 'True' or 'False' (make sure the first character is capitalized): ")
        explicit_status = str(input())
            
        #check if explicit_status is valid
        while (explicit_status != "True")  and (explicit_status != "False"):
            print("\nPlease give a valid input. Type either 'True' or 'False': ")
            explicit_status = str(input())
            
        #to convert the string to a boolean
        explicit_status = eval(explicit_status)
            
        #obtain URI of the song. Have user input the URI because numerous songs can result from the search
        print("\nPlease copy and paste the entire URI for your selected song here: ")
        uri = str(input())
            
        #to give the user a chance to re-enter the URI in case they mispelled it
        print("\nDouble check that your URI is correct. Ready to confirm? Please enter 'yes' or 'no': ")
        confirmation = str(input())
            
        #check if confirmation is valid
        while (confirmation != 'yes')  and (confirmation != 'no'):
            print("\nPlease give a valid input. Type either 'yes' or 'no': ")
            confirmation = str(input())
            
        #ask for user to input URI again if they do not confirm they want to move foward. Continue asking after every input for uri
        if confirmation == 'no':
            print("\nPlease retype your URI here: ")
            uri = str(input())
                
            print("\nDouble check that your URI is correct. Ready to confirm? Please enter 'yes' or 'no': ")
            confirmation = str(input())
                
        #edit the uri dataframe!
        uri_df.loc[missing_song_index]['explicit'] = explicit_status
        uri_df.loc[missing_song_index]['uri'] = uri
            
        print("\nThe URI/explicit status for " + data_to_clean.loc[i]['song'] + " by " + artist_to_search + " is now updated on the uri_df!")
            
    #create a dataframe with all songs that have no URI
    data_to_clean = uri_df[uri_df['uri'] == ""]
    data_to_clean.reset_index(drop=True, inplace=True)
    
    print("\nAll URI/explicit statuses are accounted for!")

In [12]:
if len(data_to_clean) > 0:
    data_cleaning_func(data_to_clean, new_uri_df)

print("Success! The entire dataframe is now updated with song URIs! Please wait for Spotify's track features to be retreived.")

Success! The entire dataframe is now updated with song URIs! Please wait for Spotify's track features to be retreived.


# Get spotipy's API Information

Spotify has a plethora of information on a song's characteristics. __The following functions/cells create a dataframe with song characteristic information on each song from the DataFrame above.__

___[Information of what each variable means can be found here](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features)___

In [13]:
def getTrackFeatures(track_id):
    """
    Returns a list of selected information from Spotify's API given a song's URI

    Parameters:
        track_id (str): a track's URI
    
    Returns
        track (list): a list of song information including it's danceability, length, tempo, etc. 
    """
    md = sp.track(track_id)
    features = sp.audio_features(track_id)
    
    #meta data
    name = md['name']
    album = md['album']['name']
    artist = md['album']['artists'][0]['name']
    release_date = md['album']['release_date']
    length = md['duration_ms']
    popularity = md['popularity']
    
    #features from the data
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    key = features[0]['key']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    mode = features[0]['mode']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']
    valence = features[0]['valence']
    
    #putting it all together
    track = [name, album, artist, release_date, length, popularity, 
             acousticness, danceability, energy, instrumentalness, key,
            liveness, loudness, mode, speechiness, 
             tempo, time_signature, valence]
    return track

In [14]:
new_track_features_list = []

[new_track_features_list.append(getTrackFeatures(new_uri_df['uri'][i]) ) for i in new_uri_df.index]

new_track_features_list

[['Guy For That (Feat. Luke Combs)',
  'Guy For That',
  'Post Malone',
  '2024-07-26',
  164011,
  84,
  0.00545,
  0.608,
  0.914,
  0,
  8,
  0.137,
  -4.116,
  1,
  0.0721,
  120.01,
  4,
  0.511],
 ['360',
  'BRAT',
  'Charli xcx',
  '2024-06-07',
  133805,
  88,
  0.646,
  0.857,
  0.62,
  0.244,
  0,
  0.0934,
  -8.13,
  1,
  0.0472,
  120.042,
  4,
  0.796],
 ['Lonely Road (with Jelly Roll)',
  'Lonely Road (with Jelly Roll)',
  'mgk',
  '2024-07-26',
  189356,
  78,
  0.0103,
  0.549,
  0.679,
  0,
  7,
  0.365,
  -5.501,
  1,
  0.0346,
  100.214,
  4,
  0.235]]

In [15]:
print("The following DataFrame gives us all Spotify's Metadata for each song from the previous dataframe")

new_api_info_df = pd.DataFrame(new_track_features_list, columns=['name', 'album', 'artist',
                                                                  'release_date', 'length', 'popularity', 'acousticness',
                                                                  'danceability', 'energy', 'instrumentalness', 'key',
                                                                  'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
                                                                  'time_signature', 'valence'])

# Set the index of new_api_info_df to match the index of new_uri_df
new_api_info_df = new_api_info_df.set_index(new_uri_df.index)

#change release_date to date format
new_api_info_df['release_date'] = pd.to_datetime(new_api_info_df['release_date'])

new_api_info_df

The following DataFrame gives us all Spotify's Metadata for each song from the previous dataframe


,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
3026,Guy For That (Feat. Luke Combs),Guy For That,Post Malone,2024-07-26,164011,84,0.00545,0.608,0.914,0.000,8,0.1370,-4.116,1,0.0721,120.010,4,0.511
3027,360,BRAT,Charli xcx,2024-06-07,133805,88,0.64600,0.857,0.620,0.244,0,0.0934,-8.130,1,0.0472,120.042,4,0.796
3028,Lonely Road (with Jelly Roll),Lonely Road (with Jelly Roll),mgk,2024-07-26,189356,78,0.01030,0.549,0.679,0.000,7,0.3650,-5.501,1,0.0346,100.214,4,0.235


## Joining all the data

Combine all the data collected/generated above with the previously created Google Sheets!

In [16]:
def final_df_creator(gsheets_df, df2gspread_wks, new_uri_df, new_api_info_df):
    # Concatenate df2gspread_wks columns with gsheets_df
    final_df = pd.concat([gsheets_df, df2gspread_wks.loc[:, 'uri':]], axis=1)
    
    # Replace NaNs with empty strings
    final_df = final_df.replace(np.nan, "")
    
    # Set index as integer
    final_df.index = final_df.index.astype(int)
    
    # Create a combined DataFrame for all new data to merge once
    new_data_df = pd.concat([
        new_uri_df[['uri', 'explicit']],
        new_api_info_df.loc[:, 'release_date':'valence']
    ], axis=1)
    
    # Merge final_df with new_data_df on index to update the columns
    final_df = pd.merge(final_df, new_data_df, left_index=True, right_index=True, how='left', suffixes=('', '_new_api_info'))
    
    # Define the list of columns to update
    columns_to_update = ['uri', 'explicit', 'release_date', 'length', 'popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']
    
    # Update columns in final_df with new data using combine_first
    for col in columns_to_update:
        final_df[col] = final_df[f'{col}_new_api_info'].combine_first(final_df[col])
    
    # Drop the unnecessary columns from the intermediate merges
    final_df.drop(columns=[f'{col}_new_api_info' for col in columns_to_update], inplace=True)
    
    # Add name for index column
    final_df.index.name = 'index'
    
    # Reset index and convert all integers/float/dates to strings
    final_df.reset_index(inplace=True)
    
    return final_df

# Example usage:
# Assuming gsheets_df, df2gspread_wks, new_uri_df, and new_api_info_df are your dataframes
final_df = final_df_creator(gsheets_df, df2gspread_wks, new_uri_df, new_api_info_df)
final_df

,index,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,current_year,peak_week_streams,2024-07-26,2024-07-19,2024-07-12,2024-07-05,2024-06-28,2024-06-21,2024-06-14,2024-06-07,2024-05-31,2024-05-24,2024-05-17,2024-05-10,2024-05-03,2024-04-26,2024-04-19,2024-04-12,2024-04-05,2024-03-29,2024-03-22,2024-03-15,2024-03-08,2024-03-01,2024-02-23,2024-02-16,2024-02-09,2024-02-02,2024-01-26,2024-01-19,2024-01-12,2024-01-05,2023-12-29,2023-12-22,2023-12-15,2023-12-08,2023-12-01,2023-11-24,2023-11-17,2023-11-10,2023-11-03,2023-10-27,2023-10-20,2023-10-13,2023-10-06,2023-09-29,2023-09-22,2023-09-15,2023-09-08,2023-09-01,2023-08-25,2023-08-18,2023-08-11,2023-08-04,2023-07-28,2023-07-21,2023-07-14,2023-07-07,2023-06-30,2023-06-23,2023-06-16,2023-06-09,2023-06-02,2023-05-26,2023-05-19,2023-05-12,2023-05-05,2023-04-28,2023-04-21,2023-04-14,2023-04-07,2023-03-31,2023-03-24,2023-03-17,2023-03-10,2023-03-03,2023-02-24,2023-02-17,2023-02-10,2023-02-03,2023-01-27,2023-01-20,2023-01-13,2023-01-06,2022-12-30,2022-12-23,2022-12-16,2022-12-09,2022-12-02,2022-11-25,2022-11-18,2022-11-11,2022-11-04,2022-10-28,2022-10-21,2022-10-14,2022-10-07,2022-09-30,2022-09-23,2022-09-16,2022-09-09,2022-09-02,2022-08-26,2022-08-19,2022-08-12,2022-08-05,2022-07-29,2022-07-22,2022-07-15,2022-07-08,2022-07-01,2022-06-24,2022-06-17,2022-06-10,2022-06-03,2022-05-27,2022-05-20,2022-05-13,2022-05-06,2022-04-29,2022-04-22,2022-04-15,2022-04-08,2022-04-01,2022-03-25,2022-03-18,2022-03-11,2022-03-04,2022-02-25,2022-02-18,2022-02-11,2022-02-04,2022-01-28,2022-01-21,2022-01-14,2022-01-07,2021-12-31,2021-12-24,2021-12-17,2021-12-10,2021-12-03,2021-11-26,2021-11-19,2021-11-12,2021-11-05,2021-10-29,2021-10-22,2021-10-15,2021-10-08,2021-10-01,2021-09-24,2021-09-17,2021-09-10,2021-09-03,2021-08-27,2021-08-20,2021-08-13,2021-08-06,2021-07-30,2021-07-23,2021-07-16,2021-07-09,2021-07-02,2021-06-25,2021-06-18,2021-06-11,2021-06-04,2021-05-28,2021-05-21,2021-05-14,2021-05-07,2021-04-30,2021-04-23,2021-04-16,2021-04-09,2021-04-02,2021-03-26,2021-03-19,2021-03-12,2021-03-05,2021-02-26,2021-02-19,2021-02-12,2021-02-05,2021-01-29,2021-01-22,2021-01-15,2021-01-08,2021-01-01,2020-12-25,2020-12-18,2020-12-11,2020-12-04,2020-11-27,2020-11-20,2020-11-13,2020-11-06,2020-10-30,2020-10-23,2020-10-16,2020-10-09,2020-10-02,2020-09-25,2020-09-18,2020-09-11,2020-09-04,2020-08-28,2020-08-21,2020-08-14,2020-08-07,2020-07-31,2020-07-24,2020-07-17,2020-07-10,2020-07-03,2020-06-26,2020-06-19,2020-06-12,2020-06-05,2020-05-29,2020-05-22,2020-05-15,2020-05-08,2020-05-01,2020-04-24,2020-04-17,2020-04-10,2020-04-03,2020-03-27,2020-03-20,2020-03-13,2020-03-06,2020-02-28,2020-02-21,2020-02-14,2020-02-07,2020-01-31,2020-01-24,2020-01-17,2020-01-10,2020-01-03,2019-12-27,2019-12-20,2019-12-13,2019-12-06,2019-11-29,2019-11-22,2019-11-15,2019-11-08,2019-11-01,2019-10-25,2019-10-18,2019-10-11,2019-10-04,2019-09-27,2019-09-20,2019-09-13,2019-09-06,2019-08-30,2019-08-23,2019-08-16,2019-08-09,2019-08-02,2019-07-26,2019-07-19,2019-07-12,2019-07-05,2019-06-28,2019-06-21,2019-06-14,2019-06-07,2019-05-31,2019-05-24,2019-05-17,2019-05-10,2019-05-03,2019-04-26,2019-04-19,2019-04-12,2019-04-05,2019-03-29,2019-03-22,2019-03-15,2019-03-08,2019-03-01,2019-02-22,2019-02-15,2019-02-08,2019-02-01,2019-01-25,2019-01-18,2019-01-11,2019-01-04,2018-12-28,2018-12-21,2018-12-14,2018-12-07,2018-11-30,2018-11-23,2018-11-16,2018-11-09,2018-11-02,2018-10-26,2018-10-19,2018-10-12,2018-10-05,2018-09-28,2018-09-21,2018-09-14,2018-09-07,2018-08-31,2018-08-24,2018-08-17,2018-08-10,2018-08-03,2018-07-27,2018-07-20,2018-07-13,2018-07-06,2018-06-29,2018-06-22,2018-06-15,2018-06-08,2018-06-01,2018-05-25,2018-05-18,2018-05-11,2018-05-04,2018-04-27,2018-04-20,2018-04-13,2018-04-06,2018-03-30,2018-03-23,2018-03-16,2018-03-09,2018-03-02,2018-02-23,2018-02-16,2018-02-09,2018-02-02,2018-01-26,2018-01-19,2018-01-12,2018-01-05,2017-12-29,2017-12-15,2017-12-08,2017-12-01,2017-11-24,2017-11-17,2017-11-10,2017-11-03,2017-10-27,2017-10-20,2017-1

## Converting the Dataframe to a Google Sheet
The codes above take a long time to run! To ease the data collection process and save time for research purposes, the dataframe is to be converted to a Google Sheet (same link with the HITS Streaming data) using the following code:

In [49]:
url = "https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit#gid=0"

spreadsheet_key = url.split("/")[-2]

wks_name = 'DF to Gspread'

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('gspread_service_account.json', scope)

gc = gspread.authorize(credentials)

d2g.upload(final_df, spreadsheet_key, wks_name, credentials = credentials,
          row_names=False) 


# backup to save
# DF to Gspread

<Worksheet 'DF to Gspread' id:2071085388>

You can now use the worksheet above for analytical purposes!! Happy analyzing! :)

__Bonus:__ The following searches up song uri info and track features for an indivudual song 

In [47]:
def song_uri_search(q):
    """
    Prints out different URIs (and other info) from a resulting search on spotipy
    
    Parameter:
        q (str): a string to look up using spotipy's search function
    """
    result = sp.search(q) 
    
    for song_info_dict in range(len(result['tracks']['items'])):
        print("Song: ", result['tracks']['items'][song_info_dict]['name'])
        print("Artist: ", result['tracks']['items'][song_info_dict]['artists'][0]['name'])
        print("Album: ", result['tracks']['items'][song_info_dict]['album']['name'])
        print("Explicit status: ", result['tracks']['items'][song_info_dict]['explicit'])
        print("URI: ", result['tracks']['items'][song_info_dict]['uri'],"\n")

song_uri_search("I Am Not Okay	Jelly Roll	I Am Not Okay")

Song:  I Am Not Okay
Artist:  Jelly Roll
Album:  I Am Not Okay
Explicit status:  False
URI:  spotify:track:76ODTQOl0JZQbhfxs6nRV9 

Song:  Save Me
Artist:  Jelly Roll
Album:  Self Medicated
Explicit status:  False
URI:  spotify:track:1rnYNAJPAb586NhetosdNW 

Song:  I Am Not Okay (Originally Performed by Jelly Roll) [Instrumental]
Artist:  Vox Freaks
Album:  I Am Not Okay (Originally Performed by Jelly Roll) [Instrumental]
Explicit status:  False
URI:  spotify:track:6UH2U2GEWq3hpLE4itLpm8 

Song:  Son Of A Sinner
Artist:  Jelly Roll
Album:  Ballads of the Broken
Explicit status:  False
URI:  spotify:track:6ngGiVX91DRmGDH4G5EgeO 

Song:  I Am Not Okay (Originally Performed By Jelly Roll) - Instrumental Version
Artist:  Troy Tha Studio Rat
Album:  I Am Not Okay (Originally Performed By Jelly Roll) [Instrumental Version]
Explicit status:  False
URI:  spotify:track:2D8qpfFEfRVOdgvtiJoXTM 

Song:  Lonely Road (with Jelly Roll)
Artist:  mgk
Album:  Lonely Road (with Jelly Roll)
Explicit statu

In [48]:
getTrackFeatures('spotify:track:76ODTQOl0JZQbhfxs6nRV9')

['I Am Not Okay',
 'I Am Not Okay',
 'Jelly Roll',
 '2024-06-12',
 197133,
 78,
 0.0602,
 0.512,
 0.672,
 0,
 7,
 0.158,
 -5.298,
 1,
 0.03,
 76.821,
 4,
 0.393]